In [1]:
from pathlib import Path
import numpy as np
import pandas as pd

In [ ]:
class CSVSequentialDataset(Dataset):
    def __init__(self, csvfile, window_size, stride):
        super(CSVSequentialDataset, self).__init__()

        self.xcols = ['input_orientation_yaw', 'input_orientation_pitch', 'input_orientation_roll',
                      'acceleration_x', 'acceleration_y', 'acceleration_z']
        self.ycols = ['input_orientation_yaw', 'input_orientation_pitch', 'input_orientation_roll']

        self.csv = pd.read_csv(csvfile)
        self.window_size = window_size
        
        self.indexes = []
        i = 0
        while i <= len(self.csv) - window_size * 2:
            self.indexes.append((i, i + window_size))
            i += stride

    def __len__(self):
        return len(self.indexes)

    def __getitem__(self, idx):
        sx, tx = self.indexes[idx]
        ty = tx + 18
        x = self.csv.iloc[sx:tx][self.xcols].to_numpy()
        y = self.csv.iloc[ty][self.ycols]

        x = torch.tensor(x, dtype=torch.float32) # 300, 6
        y = torch.tensor(y, dtype=torch.float32) # 1, 3

        return x, y

In [3]:
XCOLS = ['input_orientation_yaw', 'input_orientation_pitch', 'input_orientation_roll', 
         'acceleration_x', 'acceleration_y', 'acceleration_z']
YCOLS = ['input_orientation_yaw', 'input_orientation_pitch', 'input_orientation_roll']

In [2]:
train_files = sorted(list(Path('data/1116').glob('*scene3_0.csv')))
test_files = sorted(list(Path('data/1116').glob('*scene3_1.csv')))

In [23]:
window_size = 300
stride = 1

In [24]:
X_train = []
Y_train = []
for f in train_files:
    print(f)
    csv = pd.read_csv(f)
    i = 0
    while i <= len(csv) - window_size * 2:
        X_train.append(csv.iloc[i:i + window_size][XCOLS].to_numpy())
        Y_train.append(csv.iloc[i + window_size + 18][YCOLS].to_numpy())
        i += stride

data/1116/user1_scene3_0.csv
data/1116/user2_scene3_0.csv
data/1116/user3_scene3_0.csv
data/1116/user4_scene3_0.csv
data/1116/user5_scene3_0.csv
data/1116/user6_scene3_0.csv
data/1116/user7_scene3_0.csv


In [25]:
np.savez_compressed('data/1116/train-win_300.npz', X=X_train, Y=Y_train)

In [26]:
X_train = []
Y_train = []
for f in test_files:
    print(f)
    csv = pd.read_csv(f)
    i = 0
    while i <= len(csv) - window_size * 2:
        X_train.append(csv.iloc[i:i + window_size][XCOLS].to_numpy())
        Y_train.append(csv.iloc[i + window_size + 18][YCOLS].to_numpy())
        i += stride

data/1116/user1_scene3_1.csv
data/1116/user2_scene3_1.csv
data/1116/user3_scene3_1.csv
data/1116/user4_scene3_1.csv
data/1116/user5_scene3_1.csv
data/1116/user6_scene3_1.csv
data/1116/user7_scene3_1.csv


In [27]:
np.savez_compressed('data/1116/test-win_300.npz', X=X_train, Y=Y_train)

In [28]:
X_train[0].shape

(300, 6)